# Interface - *style-transfer-paraphrase*

In [1]:
import pickle

with open("datasets/paranmt_filtered/train.pickle", "rb") as f:
    parse_data = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'datasets/paranmt_filtered/train.pickle'

In [8]:
# Create data set
with open("../neutralizing-bias/src/bias_data/WNC/biased.full", "r", encoding="utf8") as input_file:
    with open("data_samples/wnc_biased.txt", "w", encoding="utf8") as output_file_biased:
        with open("data_samples/wnc_neutral.txt", "w", encoding="utf8") as output_file_neutral:
            for line in input_file:
                line_split = line.split("\t")
                output_file_biased.write(line_split[3] + "\n")
                output_file_neutral.write(line_split[4] + "\n")

0it [00:00, ?it/s]

In [3]:
print(parse_data[0])

('S > NP VP .', 'S > SBAR , NP VP .', False, 'Mr. Whetstone is goingto speak to you after I finish.', "after I'm done, Mr. Whetstone will be speaking.", (0.3, 0.375, 0.33333333333333326), (-0.33333333333333337, False), 2, ('en', 'en'))


In [4]:
from style_paraphrase.inference_utils import GPT2Generator

model_dir = "paraphraser_gpt2_large"
top_p_value = 0.6

paraphraser = GPT2Generator(model_dir, upper_length="same_5")

inputs = ["This is an example sentence."]

for input in inputs:
    paraphraser.modify_p(top_p=0.0)
    greedy_decoding = paraphraser.generate(input)
    print("\ngreedy sample:\n{}\n".format(greedy_decoding))
    paraphraser.modify_p(top_p=top_p_value)
    top_p_60_samples, _ = paraphraser.generate_batch([input, input, input])
    top_p_60_samples = "\n".join(top_p_60_samples)
    print("top_p = {:.2f} samples:\n{}\n".format(top_p_value, top_p_60_samples))

c:\Users\paulpommer\repos\style-transfer-paraphrase\.venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at paraphraser_gpt2_large were not used when initializing GPT2LMHeadModel: ['transformer.extra_embedding_project.weight', 'transformer.extra_embedding_project.bias']
- This IS expected if you are initializing GPT2LMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2LMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
c:\Users\paulpommer\repos\style-transfer-paraphrase


greedy sample:
it's an example sentence.

top_p = 0.60 samples:
this is an example sentence.
it's an example sentence.
this is an example sentence.

